# TCG Chatbot LLM

## 1. INTRODUCCIÓN

La finalidad de este proyecto es encontrar una necesidad o problema a resolver en nuestras vidas y a través de los conocimientos aprendidos durante el módulo, buscar los datos y las herramientas necesarias para crear un modelo que nos permita solucionar el problema en cuestión.

### 1.1. El problema

Como a la mayoría de milenials, Pokémon marcó gran parte de mi infancia, ya fuera con los videojuegos de la Gameboy, con el anime o con el juego de cartas. Pokémon estuvo presente durante muchos años en mi vida, pero con la llegada de la pubertad todo lo relacionado con Pokémon desapareció.

No obstante, mucho tiempo después, la nostalgia tocó a la puerta en forma de cartas de Pokémon TCG, reavivando el niño que llevo dentro. Desde entonces, las colecciono.

Sin embargo, antes de añadir una carta a la colección siempre es necesario llevar a cabo un proceso de investigación. Por ello, en mi día a día dedico mucho tiempo en buscar datos sobre diferentes cartas en muchas plataformas incluyendo información sobre el Pokémon, la expansión a la que pertenece, el año en que salió... y por supuesto, su precio.

### 1.2. La solución

La intención de esta práctica es crear un modelo capaz de facilitar todo el proceso de investigación, ofreciendo la información que se necesite en el momento.

De este modo, se diseñará un modelo capaz de recibir una pregunta como input referente a cualquier carta de Pokémon TCG, y el modelo será capaz de responder acertadamente dicha pregunta.

Como es lógico, lo más importante para este modelo será la base de datos, la cual se obtendrá directamente de una API de Pokémon TCG con una cantidad de datos asombrosa sobre cada carta existente: precio, expansión, número, ilustrador, año de salida, etc.

## 2. MODELO

Para llevar a cabo este proyecto se van a intentar diferentes opciones, entre ellas un modelo de reconocimiento de nombres de entidades, un modelo pequeño y barato especializado en preguntas y respuestas y un modelo generativo como es el GPT-3.5-turbo.

### 2.1. Modelo para procesar la consulta al formato de la API

* NER

En primer lugar, vamos a probar con un modelo de reconocimiento de nombre de entidades, con la esperanza de que sea capaz de reconocer los nombres de los Pokemon y de los Set.

In [ ]:
from transformers import pipeline

# Cargar un modelo pre-entrenado de lenguaje natural
nlp = pipeline("ner", grouped_entities=True, model="dbmdz/bert-large-cased-finetuned-conll03-english")

def modelo_NER(query):

    # Paso 1: Obtener las entidades reconocidas por el modelo
    entities = nlp(query)

    # Paso 2: Inicializar variables para las claves
    name_keyword = None
    set_keyword = None

    # Paso 3: Buscar las entidades clave
    for entity in entities:
        if entity["entity_group"] == "MISC":  # Podría ser el nombre del set
            set_keyword = entity["word"].lower()
        elif entity["entity_group"] == "ORG":  # Podría ser el nombre del Pokémon
            name_keyword = entity["word"].lower()

    # Paso 4: Construir la query para la API
    api_query = []
    if name_keyword:
        api_query.append(f"name:{name_keyword}")
    if set_keyword:
        api_query.append(f"set.name:{set_keyword}")

    return " ".join(api_query) if api_query else None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


* Ejemplo 1

In [ ]:
ejemplo = nlp("What are the cards from Charizard in Team Rocket")

In [ ]:
ejemplo

[{'entity_group': 'ORG',
  'score': 0.913363,
  'word': 'Charizard',
  'start': 24,
  'end': 33},
 {'entity_group': 'ORG',
  'score': 0.90259176,
  'word': 'Team Rocket',
  'start': 37,
  'end': 48}]

* Ejemplo 2

In [ ]:
ejemplo2 = nlp("I want to know the price of the Neo Genesis Lugia")

In [ ]:
ejemplo2

[{'entity_group': 'MISC',
  'score': 0.9893898,
  'word': 'Neo Genesis Lugia',
  'start': 32,
  'end': 49}]

* Ejemplo 3

In [ ]:
ejemplo3 = nlp("Tell me information about the Umbreon from Evolving Skies")

In [ ]:
ejemplo3

[{'entity_group': 'MISC',
  'score': 0.8483124,
  'word': 'Um',
  'start': 30,
  'end': 32},
 {'entity_group': 'PER',
  'score': 0.27535868,
  'word': '##bre',
  'start': 32,
  'end': 35},
 {'entity_group': 'MISC',
  'score': 0.6977882,
  'word': '##on from Evolving Skies',
  'start': 35,
  'end': 57}]

En el primer ejemplo, ha reconocido correctamente tanto el nombre del Pokémon como el nombre de la expansión (a pesar de ser un nombre compuesto). Sin embargo, los ha reconocido ambos como "ORG". Además, cuando intentamos ejemplos más complejos, el modelo no funciona como se esperaba. En el ejemplo 2 toma como un mismo nombre el conjunto de la expansión y el Pokémon. Y en el ejemplo 3 directamente no reconoce ninguno de los nombres.

En resumen, este modelo nos podría servir de ayuda para búsquedas más sencillas, o mejorarlo con un Fine Tuning utilizando una base de datos con información sobre nombres de Pokémon y expansiones. No obstante, en este proyecto se intentará conseguir mejores resultados mediante otras alternativas.

* Modelo GPT y Prompt Engineering

En segundo lugar, se decide optar por otra alternativa, un LLM chatbot capaz de analizar lenguaje natural de una forma más efectiva. En este caso se usará el GPT-3.5-turbo, con el objetivo de que sea capaz de reconocer los nombres de los Pokémon, expansiones u otros datos importantes.

Además, mediante prompt engineering se le asigna un rol, una tarea, un formato de salida y varios ejemplos para obtener el resultado requerido.

In [ ]:
from openai import OpenAI
import json
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client = OpenAI(api_key = api_key)

Enter your OpenAI API Key:··········


In [ ]:
def input_format(question, model="gpt-3.5-turbo", temperature=0):

    prompt_template = """
**DON'T MAKE UP INFORMATION**. Extract specific details **ONLY** from the user's question and return a valid JSON dictionary.

### **Posible Keys ONLY IF GIVEN**:
- name
- number
- set.name
- artist

### **Examples**:

1. User: Who designed the neo genesis lugia and how much is it in dollars?
Solution: {{"name":"lugia", "set.name":"neo genesis"}}

2. User: What is the price of the umbreons from evolving skies? order them from low to high prices
Solution: {{"name":"umbreon", "subtypes":"vmax", "set.name":"evolving skies"}}

3. User: when was released the charizard 199? can you show me a photo?
Solution: {{"name":"charizard", "number":"199"}}

3. User: show me a photo of the drowzee desgined by Komiya?
Solution: {{"name":"drowzee", "artist":"Komiya"}}

### New user Input:
{question}

### Your solution:
"""
    prompt = prompt_template.format(question=question)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    str_dict = response.choices[0].message.content
    json_dict = json.loads(str_dict)
    return json_dict  # Devuelve el diccionario en formato JSON


* Ejemplo 1

In [ ]:
question_1 = "who designed the houndoom from neo revelation?"

In [ ]:
response_1 = input_format(question_1)
print(response_1)

{'name': 'houndoom', 'set.name': 'neo revelation'}


* Ejemplo 2

In [ ]:
question_2 = "how much is it the unseen forces celebi?"

In [ ]:
response_2 = input_format(question_2)
print(response_2)

{'name': 'celebi', 'set.name': 'unseen forces'}


* Ejemplo 3

In [ ]:
question_3 = "when was released the charizard from base set?"

In [ ]:
response_3 = input_format(question_3)
print(response_3)

{'name': 'charizard', 'set.name': 'base'}


* Ejemplo 4

In [ ]:
question_4 = "What is the price in US market of the different umbreons from evolving skies? Please order them from cheapest to most expensive"

In [ ]:
response_4 = input_format(question_4)
print(response_4)

{'name': 'umbreon', 'set.name': 'evolving skies'}


Como se puede comprobar el comportamiento del modelo es más que aceptable, aunque no perfecto, pero la mayoría de las veces cumple su función.

No obstante, la API no entiende el formato JSON.Además, las palabras compuestas como Team Rocket deben estar agrupadas entre \" para que la API las considere como un mismo nombre.

Para llevar a cabo dicha transformación, se diseña una función.

In [ ]:
def api_format(json_dict):

    formatted_parts = []

    for key, value in json_dict.items():
        formatted_parts.append(f'{key}:"{value}"')  # Agregar \" al inicio y final del valor

    return " ".join(formatted_parts)  # Unir todos los elementos con espacio

* Ejemplo 1

In [ ]:
api_format_1 = api_format(response_1)
print(api_format_1)

name:"houndoom" set.name:"neo revelation"


* Ejemplo 2

In [ ]:
api_format_2 = api_format(response_2)
api_format_2

'name:"celebi" set.name:"unseen forces"'

* Ejemplo 3

In [ ]:
api_format_3 = api_format(response_3)
api_format_3

'name:"charizard" set.name:"base"'

* Ejemplo 4

In [ ]:
api_format_4 = api_format(response_4)
api_format_4

'name:"umbreon" set.name:"evolving skies"'

### 2.2. Función para llamar a la API

Tal y como se explicó con anterioridad, se va a utilizar un API de Pokemon TCG con gran cantidad de datos sobre todas las cartas existentes con una información muy completa, estructurada y de fácil acceso.

In [ ]:
import requests

def api_card_data(query):
    url = "https://api.pokemontcg.io/v2/cards"
    headers = {"X-Api-Key": "12bf9d80-fc20-494b-8445-a1360e7ce3c8"}
    params = {"q": query}
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API Error: {response.status_code}"}

* Ejemplo 1

In [ ]:
card_data_1 = api_card_data(api_format_1)
print(card_data_1)

{'data': [{'id': 'neo3-8', 'name': 'Houndoom', 'supertype': 'Pokémon', 'subtypes': ['Stage 1'], 'level': '35', 'hp': '70', 'types': ['Darkness'], 'evolvesFrom': 'Houndour', 'attacks': [{'name': 'Dark Flame', 'cost': ['Fire'], 'convertedEnergyCost': 1, 'damage': '20', 'text': 'Discard 1 Fire Energy card attached to Houndoom or this attack does nothing. If there are any Darkness Energy cards in your discard pile, attach 1 of them to Houndoom.'}, {'name': 'Black Fang', 'cost': ['Darkness', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '30+', 'text': 'Flip a number of coins equal to the number of Darkness Energy attached to Houndoom. This attack does 30 damage plus 10 more damage for each heads.'}], 'resistances': [{'type': 'Psychic', 'value': '-30'}], 'retreatCost': ['Colorless'], 'convertedRetreatCost': 1, 'set': {'id': 'neo3', 'name': 'Neo Revelation', 'series': 'Neo', 'printedTotal': 64, 'total': 66, 'legalities': {'unlimited': 'Legal'}, 'ptcgoCode': 'N3', 'releaseDate

* Ejemplo 2

In [ ]:
card_data_2 = api_card_data(api_format_2)
print(card_data_2)


{'data': [{'id': 'ex10-117', 'name': 'Celebi ex', 'supertype': 'Pokémon', 'subtypes': ['Basic', 'ex'], 'hp': '70', 'types': ['Grass'], 'rules': ['When Pokémon-ex has been Knocked Out, your opponent takes 2 Prize cards.'], 'attacks': [{'name': 'Spiral Leaf', 'cost': ['Grass'], 'convertedEnergyCost': 1, 'damage': '', 'text': "Flip a coin. If heads, put 1 damage counter on each of your opponent's Pokémon. If tails, remove 1 damage counter from each of your Pokémon."}, {'name': 'Time Trap', 'cost': ['Grass', 'Colorless'], 'convertedEnergyCost': 2, 'damage': '30', 'text': "Flip a coin. If heads, look at the top 4 cards of your opponent's deck, and put them back on top of your opponent's deck in any order. If tails, look at the top 4 cards of your deck, and put them back on top of your deck in any order."}], 'weaknesses': [{'type': 'Fire', 'value': '×2'}], 'retreatCost': ['Colorless'], 'convertedRetreatCost': 1, 'set': {'id': 'ex10', 'name': 'Unseen Forces', 'series': 'EX', 'printedTotal': 1

* Ejemplo 3

In [ ]:
card_data_3 = api_card_data(api_format_3)
print(card_data_3)

{'data': [{'id': 'base4-4', 'name': 'Charizard', 'supertype': 'Pokémon', 'subtypes': ['Stage 2'], 'level': '76', 'hp': '120', 'types': ['Fire'], 'evolvesFrom': 'Charmeleon', 'abilities': [{'name': 'Energy Burn', 'text': "As often as you like during your turn (before your attack), you may turn all Energy attached to Charizard into Fire Energy for the rest of the turn. This power can't be used if Charizard is Asleep, Confused, or Paralyzed.", 'type': 'Pokémon Power'}], 'attacks': [{'name': 'Fire Spin', 'cost': ['Fire', 'Fire', 'Fire', 'Fire'], 'convertedEnergyCost': 4, 'damage': '100', 'text': 'Discard 2 Energy cards attached to Charizard in order to use this attack.'}], 'weaknesses': [{'type': 'Water', 'value': '×2'}], 'resistances': [{'type': 'Fighting', 'value': '-30'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless'], 'convertedRetreatCost': 3, 'set': {'id': 'base4', 'name': 'Base Set 2', 'series': 'Base', 'printedTotal': 130, 'total': 130, 'legalities': {'unlimited': 'Legal'}

* Ejemplo 4

In [ ]:
card_data_4 = api_card_data(api_format_4)
print(card_data_4)

{'data': [{'id': 'swsh7-94', 'name': 'Umbreon V', 'supertype': 'Pokémon', 'subtypes': ['Basic', 'V', 'Single Strike'], 'hp': '200', 'types': ['Darkness'], 'rules': ['V rule: When your Pokémon V is Knocked Out, your opponent takes 2 Prize cards.'], 'attacks': [{'name': 'Mean Look', 'cost': ['Darkness'], 'convertedEnergyCost': 1, 'damage': '30', 'text': "During your opponent's next turn, the Defending Pokémon can't retreat."}, {'name': 'Moonlight Blade', 'cost': ['Darkness', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '80+', 'text': 'If this Pokémon has any damage counters on it, this attack does 80 more damage.'}], 'weaknesses': [{'type': 'Grass', 'value': '×2'}], 'retreatCost': ['Colorless', 'Colorless'], 'convertedRetreatCost': 2, 'set': {'id': 'swsh7', 'name': 'Evolving Skies', 'series': 'Sword & Shield', 'printedTotal': 203, 'total': 237, 'legalities': {'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}, 'ptcgoCode': 'EVS', 'releaseDate': '2021/08/27'

### 2.3. Función para estructurar y limpiar los datos de la API

Los datos obtenidos de la API son en formato diccionario JSON y la información es abundante y gran parte de ella es innecesaria. Esta gran cantidad de datos sólo confundiría al modelo de GPT-3.5-turbo encargado de responder la pregunta. Por ello, se diseña una función cuya finalidad es la de limpiar y estructurar correctamente los datos para el contexto del modelo.

In [ ]:
def context_format(card_data, max_chars=4000):
    if 'data' not in card_data or not card_data['data']:
        return "No cards found."

    context = ""
    for i, card in enumerate(card_data['data'], start=1):
        card_info = (
            f"Card {i}:\n"
            f"Name: {card.get('name', 'N/A')}\n"
            f"Subtypes: {', '.join(card.get('subtypes', []))}\n"
            f"Number: {card.get('number', 'N/A')}\n"
            f"Artist: {card.get('artist', 'N/A')}\n"
            f"Set Name: {card.get('set', {}).get('name', 'N/A')}\n"
            f"Set Series: {card.get('set', {}).get('series', 'N/A')}\n"
            f"Release Date: {card.get('set', {}).get('releaseDate', 'N/A')}\n"
            f"Large Image: {card.get('images', {}).get('large', 'N/A')}\n"
            f"Market Price TCGPlayer: ${card.get('tcgplayer', {}).get('prices', {}).get('holofoil', {}).get('market', 'N/A')}\n"
            f"TCGPlayer URL: {card.get('tcgplayer', {}).get('url', 'N/A')}\n"
            f"Market Price Cardmarket: {card.get('cardmarket', {}).get('prices', {}).get('avg7', 'N/A')}€\n"
            f"Cardmarket URL: {card.get('cardmarket', {}).get('url', 'N/A')}\n\n"
        )

        if len(context) + len(card_info) > max_chars:
            context += f"\nLimit reached. {len(card_data['data']) - i + 1} cards not included."
            break

        context += card_info

    return context.strip()

* Ejemplo 1

In [ ]:
context_1 = context_format(card_data_1)
print(context_1)

Card 1:
Name: Houndoom
Subtypes: Stage 1
Number: 8
Artist: Mitsuhiro Arita
Set Name: Neo Revelation
Set Series: Neo
Release Date: 2001/09/21
Large Image: https://images.pokemontcg.io/neo3/8_hires.png
Market Price TCGPlayer: $N/A
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/neo3-8
Market Price Cardmarket: 48.03€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/neo3-8


* Ejemplo 2

In [ ]:
context_2 = context_format(card_data_2)
print(context_2)

Card 1:
Name: Celebi ex
Subtypes: Basic, ex
Number: 117
Artist: Ryo Ueda
Set Name: Unseen Forces
Set Series: EX
Release Date: 2005/08/01
Large Image: https://images.pokemontcg.io/ex10/117_hires.png
Market Price TCGPlayer: $399.99
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/ex10-117
Market Price Cardmarket: 216.67€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/ex10-117


* Ejemplo 3

In [ ]:
context_3 = context_format(card_data_3)
print(context_3)

Card 1:
Name: Charizard
Subtypes: Stage 2
Number: 4
Artist: Mitsuhiro Arita
Set Name: Base Set 2
Set Series: Base
Release Date: 2000/02/24
Large Image: https://images.pokemontcg.io/base4/4_hires.png
Market Price TCGPlayer: $292.83
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/base4-4
Market Price Cardmarket: 218.28€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/base4-4

Card 2:
Name: Charizard
Subtypes: Stage 2
Number: 4
Artist: Mitsuhiro Arita
Set Name: Base
Set Series: Base
Release Date: 1999/01/09
Large Image: https://images.pokemontcg.io/base1/4_hires.png
Market Price TCGPlayer: $412.86
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/base1-4
Market Price Cardmarket: 1631.43€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/base1-4

Card 3:
Name: Charizard
Subtypes: Stage 2
Number: 6
Artist: Hiromichi Sugiyama
Set Name: Expedition Base Set
Set Series: E-Card
Release Date: 2002/09/15
Large Image: https://images.pokemontcg.io/ecard1/6_hires.png
Market Pr

In [ ]:
context_4 = context_format(card_data_4)
print(context_4)

Card 1:
Name: Umbreon V
Subtypes: Basic, V, Single Strike
Number: 94
Artist: 5ban Graphics
Set Name: Evolving Skies
Set Series: Sword & Shield
Release Date: 2021/08/27
Large Image: https://images.pokemontcg.io/swsh7/94_hires.png
Market Price TCGPlayer: $2.76
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/swsh7-94
Market Price Cardmarket: 3.75€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/swsh7-94

Card 2:
Name: Umbreon V
Subtypes: Basic, V, Single Strike
Number: 189
Artist: Teeziro
Set Name: Evolving Skies
Set Series: Sword & Shield
Release Date: 2021/08/27
Large Image: https://images.pokemontcg.io/swsh7/189_hires.png
Market Price TCGPlayer: $247.84
TCGPlayer URL: https://prices.pokemontcg.io/tcgplayer/swsh7-189
Market Price Cardmarket: 212.91€
Cardmarket URL: https://prices.pokemontcg.io/cardmarket/swsh7-189

Card 3:
Name: Umbreon VMAX
Subtypes: VMAX, Single Strike
Number: 95
Artist: AKIRA EGAWA
Set Name: Evolving Skies
Set Series: Sword & Shield
Release Date: 2021/0

### 2.4. RAG

A continuación, aplicamos Retrieval Augmented Generation, tomando los datos de la API para alimentar el contexto del modelo seleccionado y así poder contestar a la pregunta con mayor precisión.

En primer lugar, se seleccionó un modelo especializado en question-answer, concretamente el "distilbert-base-uncased-distilled-squad". No obstante, debido a su mal rendimiento y escaso entendimiento del contexto, de nuevo se tuvo que optar por un modelo más potente como es el "GPT-3.5-turbo".

In [ ]:
def answer_question(context,question, model="gpt-3.5-turbo", temperature = 0):

    prompt_template = """
    You are a Pokémon TCG expert. You must answer the given question and extract the information ONLY from the given context. If you cannot answer the question with the context, please respond with 'I don't know'.

    ### Context:
    {context}

    ### Question:
    {question}

    ### Answer:
    """
    prompt = prompt_template.format(question=question, context=context)
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

* Ejemplo 1

In [ ]:
answer_1 = answer_question(context_1, question_1)
print(answer_1)

Mitsuhiro Arita


* Ejemplo 2

In [ ]:
answer_2 = answer_question(context_2, question_2)
print(answer_2)

The market price for the Unseen Forces Celebi is $399.99 on TCGPlayer and 216.67€ on Cardmarket.


* Ejemplo 3

In [ ]:
answer_3 = answer_question(context_3, question_3)
print(answer_3)

The Charizard from the Base Set was released on 1999/01/09.


* Ejemplo 4

In [ ]:
answer_4 = answer_question(context_4, question_4)
print(answer_4)

The prices in the US market of the different Umbreons from Evolving Skies, ordered from cheapest to most expensive, are as follows:
1. Umbreon V - $2.76
2. Umbreon V - $34.34
3. Umbreon VMAX - $15.54
4. Umbreon VMAX - $49.65
5. Umbreon VMAX - $1527.75


## 3. PIPELINE


In [ ]:
def main(question):

    # 1. Procesar la entrada al formato adecuado para la API
    query_json = input_format(question)
    query = api_format(query_json)

    # 2. Llamar a la API para obtener la información solicitada
    card_data = api_card_data(query)

    # 3. Estructurar y filtrar el contexto
    context = context_format(card_data)

    # 4. Generar una respuesta con el modelo QA usando RAG
    answer = answer_question(context, question)

    # 5. Mostrar la salida
    print(answer)

#if __name__ == "__main__":
#    main()

## 4. PoC


Para comprobar el correcto funcionamiento del modelo, se va a realizar una prueba de concepto con diferentes preguntas de entrada y verificar sus respuestas.

In [ ]:
question1 = "What is the price in US market of the different charizards from generations? Please order them from cheapest to most expensive"
answer1 = main(question1)
answer1

$21.51 for Charizard, $16.87 for Charizard-EX, $64.9 for M Charizard-EX


In [ ]:
question2 = "who designed the lugia from neo genesis? show me a photo"
answer2 = main(question2)
answer2

The Lugia from Neo Genesis was designed by Hironobu Yoshida. You can view a photo of the card here: [Lugia from Neo Genesis](https://images.pokemontcg.io/neo1/9_hires.png)


In [ ]:
question3 = "when was released the rayquaza from dragons exalted?"
answer3 = main(question3)
answer3

The Rayquaza from Dragons Exalted was released on 2012/08/15.


In [ ]:
question4 = "Show me a photo of the most expensive Gengar in fusion strike"
answer4 = main(question4)
answer4

![Gengar VMAX](https://images.pokemontcg.io/swsh8/271_hires.png)


In [ ]:
question5 = "Is there any giratina designed by shinji kanda? can you show me a photo?"
answer5 = main(question5)
answer5

Yes, there is a Giratina designed by Shinji Kanda. You can see a photo of it by clicking on this link: [Giratina V](https://images.pokemontcg.io/swsh11/186_hires.png)


In [ ]:
question6 = "whats the most expensive lugia from silver tempest in euros?"
answer6 = main(question6)
answer6

The most expensive Lugia from Silver Tempest in euros is Card 5: Lugia V with a market price of 300.15€ on Cardmarket.


Los resultados obtenidos del modelo son más que aceptables, cumple perfectamente su función y el objetivo para el que fue diseñado. El primer modelo demuestra ser perfectamente capaz de reconocer las palabras clave de la pregunta y formatearlas para la llamada a la API. Y el segundo modelo también demuestra comprender a la perfección las preguntas realizadas y contestarlas en consecuencia.

No obstante, a pesar de cumplir el objetivo buscado, el modelo no es perfecto. Una característica importante a mejorar sería la capacidad de comparar varios resultados, ya que en el modelo actual sólo se puede llamar a la API una vez por pregunta, y debido a la estructura de la API, es imposible comparar dos cartas de diferente nombre, set o ilustrador, lo cual sería muy interesante.

In [ ]:
question7 = "which one is more expensive the charizard ex 199 or the tyranitar from unseen forces?"
answer7 = main(question7)
answer7

I don't know.


Otra posible característica a mejorar sería añadir una memoria al modelo, ya que esta capacidad también permitiría comparar nuevas búsquedas con búsquedas anteriores.

En definitiva, el modelo tiene muchos puntos a mejorar, pero con las herramientas y el tiempo del que se dispone, se ha obtenido un modelo perfectamente funcional y que cumple su objetivo principal: facilitar la búsqueda de información sobre cartas de Pokémon TCG.

A continuación, en el siguiente documento se lleva a cabo el despliegue del modelo en Streamlit para facilitar el uso y la aplicación del modelo.